In [3]:
# korea univ morpheme analyzer
sentence = '이를 위해서도 독도가 일본 영토라는 억지 주장이나 군국주의 망령에 사로잡힌 야스쿠니 신사참배 같은 근시안적인 선거공약을 마련해서는 안될 것이다.'
sentence = '실제로 대우가 현재 분양중인 일산 대우1차아파트의 37평형에 관한 팩스정보를 얻으려면 일반 전화기로 02-759-3841∼2를 돌린 뒤 "01(분양안내), 02(분양금납부안내), 03(분양금입금안내), 04(분양정보팩스전송서비스)중 하나를 선택하라"는 음성이 흘러 나오면 04를 선택한다.'
sentence = '4월7~8일 : 미국영화협회(MPAA) 제프리 하디 아시아태평양 담당 부회장이 문화관광부와 산업자원부를 방문해 "스크린쿼터 완화할 경우 5억달러 정도를 투자해 전국 20개 지역에 스크린 10개짜리 멀티플렉스를 짓겠다"고 제의.'

m_command = 'cd data/kmat/bin/;./kmat <<<\"' + sentence + '\" 2>/dev/null'
result = subprocess.check_output(m_command.encode(encoding='cp949', errors='ignore'), shell=True,
                                 executable='/bin/bash', )

mor_name_lists = []
mor_tags_lists = []

for each in result.decode(encoding='cp949', errors='ignore').split('\n'):
    if len(each) > 0:
        try:
            mor_texts = each.split('\t')[1]
        except:
            print(each)
        mor_results = mor_texts.split('+')
        
        for each_mor in mor_results:
            mor_name_lists.append(each_mor.split('/')[0])
            mor_tags_lists.append(each_mor.split('/')[1])

In [1]:
from data_utils import get_trimmed_glove_vectors, load_vocab, \
    get_processing_word, Data
from config import Config
from cnn_lstm_crf import CnnLstmCrfModel
import subprocess

def get_mor_result(sentence):
    # korea univ morpheme analyzer
    m_command = "cd data/kmat/bin/;./kmat <<<\'" + sentence + "\' 2>/dev/null"
    result = subprocess.check_output(m_command.encode(encoding='cp949', errors='ignore'), shell=True,
                                     executable='/bin/bash')
    mor_name_lists = []
    mor_tags_lists = []

    for each in result.decode(encoding='cp949', errors='ignore').split('\n'):
        if len(each) > 0:
            try:
                mor_texts = each.split('\t')[1]
            except:
                print(each)
            mor_results = mor_texts.split('+')

            for each_mor in mor_results:
                mor_name_lists.append(each_mor.split('/')[0])
                mor_tags_lists.append(each_mor.split('/')[1])

    return mor_name_lists, mor_tags_lists

In [1]:
from data_utils import get_trimmed_glove_vectors, load_vocab, \
    get_processing_word, Data
from config import Config
from cnn_lstm_crf import CnnLstmCrfModel

config = Config()

vocab_words = load_vocab(config.words_filename)
vocab_mor_tags = load_vocab(config.mor_tags_filename)
vocab_tags  = load_vocab(config.tags_filename)
vocab_chars = load_vocab(config.chars_filename)
vocab_lex_tags = load_vocab(config.lex_tags_filename)

# get processing functions
processing_word = get_processing_word(vocab_words, vocab_chars,
                lowercase=True, chars=config.chars)
processing_mor_tag = get_processing_word(vocab_mor_tags, lowercase=False)
processing_tag = get_processing_word(vocab_tags,
                lowercase=False)
processing_lex_tag = get_processing_word(vocab_lex_tags, lowercase=False)


# get pre trained embeddings
embeddings = get_trimmed_glove_vectors(config.trimmed_filename)

# create dataset
dev   = Data(config.dev_filename, processing_word, processing_mor_tag,
             processing_lex_tag, processing_tag, config.max_iter)
test  = Data(config.test_filename, processing_word, processing_mor_tag,
             processing_lex_tag, processing_tag, config.max_iter)
train = Data(config.train_filename, processing_word, processing_mor_tag,
             processing_lex_tag, processing_tag, config.max_iter)


cnn_model = CnnLstmCrfModel(config, embeddings, ntags=len(vocab_tags),nchars=len(vocab_chars))
cnn_model.build()

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [2]:
cnn_model.write_tag_result(vocab_tags, processing_word, processing_mor_tag, processing_lex_tag)

INFO:tensorflow:Restoring parameters from results/full_cnn_with_mor_lex_new_data_0924/model_fil_2345_600_100_128_drop_0.8_0924.weights


Restoring parameters from results/full_cnn_with_mor_lex_new_data_0924/model_fil_2345_600_100_128_drop_0.8_0924.weights


In [ ]:
cnn_model.interactive_shell(vocab_tags, processing_word, processing_mor_tag, processing_lex_tag)

In [2]:
import subprocess

In [3]:
input_text = '이를 위해서도 독도가 일본 영토라는 억지 주장이나 군국주의 망령에 사로잡힌 야스쿠니 신사참배 같은 근시안적인 선거공약을 마련해서는 안될 것이다.'
m_command = 'cd data/kmat/bin/;./kmat <<<\"' + input_text + '\" 2>/dev/null'
result = subprocess.check_output(m_command.encode(encoding='cp949',errors='ignore'), shell=True, executable='/bin/bash',)

In [4]:
print(result.decode(encoding='cp949',errors='ignore'))

이를	이/NP+를/JKO
위해서도	위하/VV+아서/EM+도/JX
독도가	독도/NNP+가/JKS
일본	일본/NNP
영토라는	영토/NNG+이/VCP+라는/ETM
억지	억지/NNG
주장이나	주장/NNG+이나/JKB
군국주의	군국주의/NNG
망령에	망령/NNG+에/JKB
사로잡힌	사로잡히/VV+ㄴ/ETM
야스쿠니	야스쿠니/NNP
신사참배	신사/NNG+참배/NNG
같은	같/VA+은/ETM
근시안적인	근시안적/NNG+이/VCP+ㄴ/ETM
선거공약을	선거/NNG+공약/NNG+을/JKO
마련해서는	마련/NNG+하/XSV+아서/EM+는/JX
안될	안/MAG+되/XSV+ㄹ/ETM
것이다.	것/NNB+이/VCP+다/EM+./SF




In [5]:
mor_lists = []
for each in result.decode(encoding='cp949',errors='ignore').split('\n'):
    if len(each) > 0:
        try:
            mor_text = each.split('\t')[1]
        except:
            print(each)
        mor_results = mor_text.split('+')

        for each_mor in mor_results:
            mor_lists.append(each_mor.split('/')[0])

In [6]:
mor_lists

['이',
 '를',
 '위하',
 '아서',
 '도',
 '독도',
 '가',
 '일본',
 '영토',
 '이',
 '라는',
 '억지',
 '주장',
 '이나',
 '군국주의',
 '망령',
 '에',
 '사로잡히',
 'ㄴ',
 '야스쿠니',
 '신사',
 '참배',
 '같',
 '은',
 '근시안적',
 '이',
 'ㄴ',
 '선거',
 '공약',
 '을',
 '마련',
 '하',
 '아서',
 '는',
 '안',
 '되',
 'ㄹ',
 '것',
 '이',
 '다',
 '.']